In [29]:
import pandas as pd
import numpy as np
import nltk
import re
import os
from nltk.stem.snowball import SnowballStemmer
#import datetime
#import math
#from gensim import models, corpora

In [23]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Preparing the corpus

### Stopwords, stemming and tokenizing
#### 1. load list of English stop words

In [27]:
stopwords = nltk.corpus.stopwords.words('english')

In [28]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

Import the Snowball Stemmer which is actually part of NLTK
<br>We should do **stemming** - breaking a word down into its root.</br>

In [30]:
stemmer = SnowballStemmer("english")

In [25]:
email_receivers_file = os.path.join('hillary-clinton-emails', 'EmailReceivers.csv')
emails_file = os.path.join('hillary-clinton-emails', 'Emails.csv')
persons_file = os.path.join('hillary-clinton-emails', 'Persons.csv')

In [3]:
email_receivers_df = pd.read_csv(email_receivers_file)
email_receivers_df.head(1)

,Id,EmailId,PersonId
0,1,1,80


In [4]:
emails_df = pd.read_csv(emails_file)
emails_df.head(2)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...


In [5]:
#emails_df.columns.values

In [6]:
topic_modeling_df = emails_df.copy()
topic_modeling_df.drop(['Id'
                        , 'DocNumber'
                        , 'MetadataTo'
                        , 'MetadataFrom'
                        , 'SenderPersonId'
                        , 'MetadataDateSent'
                        , 'MetadataDateReleased'
                        , 'MetadataPdfLink'
                        , 'MetadataCaseNumber'
                        , 'ExtractedTo'
                        , 'ExtractedFrom'
                        , 'ExtractedCc'
                        , 'ExtractedDateSent'
                        , 'ExtractedCaseNumber'
                        , 'ExtractedDocNumber'
                        , 'ExtractedDateReleased'
                        , 'ExtractedReleaseInPartOrFull'], axis=1, inplace=True)
topic_modeling_df.head()

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText
0,WOW,HRC_Email_296,FW: Wow,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,CHRIS STEVENS,HRC_Email_296,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
3,CAIRO CONDEMNATION - FINAL,HRC_Email_296,FVV: Cairo Condemnation - Final,NaN,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


In [7]:
topic_modeling_df.shape

(7945, 5)

In [8]:
topic_modeling_df = topic_modeling_df.dropna(subset=['ExtractedBodyText'], how='all')

In [9]:
topic_modeling_df.head()

,MetadataSubject,MetadataDocumentClass,ExtractedSubject,ExtractedBodyText,RawText
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...",UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
2,CHRIS STEVENS,HRC_Email_296,Re: Chris Stevens,Thx,UNCLASSIFIED\r\nU.S. Department of State\r\nCa...
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...
5,MEET THE RIGHT-WING EXTREMIST BEHIND ANTI-MUSL...,HRC_Email_296,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...,B6\r\nUNCLASSIFIED\r\nU.S. Department of State...
7,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,HRC_Email_296,NaN,"H <hrod17@clintonemail.corn>\r\nFriday, March ...",B6\r\nUNCLASSIFIED\r\nU.S. Department of State...


In [10]:
topic_modeling_dict = topic_modeling_df.to_dict()

In [11]:
extracted_body_text_list = list(topic_modeling_df['ExtractedBodyText'])
len(extracted_body_text_list)

6742

In [12]:
extracted_body_text_list[:2]

['B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: Latest How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\r\nMarch 3, 2011\r\nFor: Hillary',
 'Thx']

In [13]:
# remove one-word texts
extracted_body_text_list = list(filter(lambda body: ' ' in body, extracted_body_text_list))
len(extracted_body_text_list)

5938

In [109]:
extracted_body_text_list[:2]

['B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: Latest How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\r\nMarch 3, 2011\r\nFor: Hillary',
 'H <hrod17@clintonemail.com>\r\nFriday, March 11, 2011 1:36 PM\r\nHuma Abedin\r\nFw: H: Latest: How Syria is aiding Qaddafi and more... Sid\r\nhrc memo syria aiding libya 030311.docx\r\nPis print.']

In [110]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [111]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [112]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

In [130]:
filter_rules = ['<.*>'
          , '^STATE DEPT .* STATE-[0-9A-Z]+'
          , 'Date: .* \\n'
          , 'Doc No.: .* \\n'
          , 'Case No.: .* \\n'
          , 'Sent: .* \\n'
          , 'To: .* \\n'
          , '^[A-Z] .* @ .* \\n'
          , '[^a-z] :.[A-Z]'
          , '[A-Z]{1}[a-z]+, [A-Z]{1}[a-z]+ \d+, [0-9]+ \d+:\d+ [A-Z]{2}'
          , '^[a-zA-Z]{1,2}$']

def apply_filters(series):
    for rule in filter_rules:
        series = series.apply(lambda x: re.sub(rule, '', x))
    return series

In [132]:
extracted_body_series = topic_modeling_df['ExtractedBodyText']

In [119]:
extracted_body_series.size

6742

In [120]:
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
months = ['january'
          , 'february'
          , 'march'
          , 'april'
          , 'may'
          , 'june'
          , 'july'
          , 'august'
          , 'september'
          , 'october'
          , 'november'
          , 'december']

In [134]:
extracted_body_series = apply_filters(extracted_body_series)

In [169]:
from nltk.stem.porter import *
stemmer1 = PorterStemmer()

In [172]:
def tokens_from_text(text):
    tokens = tokenize_only(text)
    tokens = [word for word in tokens if word not in stopwords]
    tokens = list(filter(lambda w: w not in days, tokens))
    tokens = list(filter(lambda w: w not in months, tokens))
    tokens = [word for word in tokens if word in english_vocab]
    return [stemmer1.stem(t) for t in tokens]

In [173]:
filtered_tokens = [tokens_from_text(text) for ind, text in extracted_body_series.iteritems()]

In [174]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(filtered_tokens)

In [175]:
#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

In [176]:
#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(token_list) for token_list in filtered_tokens]

In [212]:
%time lda = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=100)
    #corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

Wall time: 21min 42s


In [213]:
# Each topic has a set of words that defines it, along with a certain probability.
lda.show_topics()

[(0,
  '0.007*"would" + 0.007*"said" + 0.006*"new" + 0.006*"presid" + 0.006*"one" + 0.005*"peopl" + 0.005*"govern" + 0.005*"time" + 0.004*"work" + 0.004*"polit"'),
 (1,
  '0.047*"secretari" + 0.044*"depart" + 0.044*"state" + 0.040*"offic" + 0.024*"meet" + 0.019*"room" + 0.015*"hous" + 0.014*"arriv" + 0.013*"rout" + 0.012*"privat"'),
 (2,
  '0.021*"call" + 0.017*"h" + 0.013*"see" + 0.013*"get" + 0.011*"work" + 0.010*"w" + 0.010*"know" + 0.009*"tomorrow" + 0.009*"go" + 0.009*"today"')]

In [214]:
lda.show_topics()

[(0,
  '0.007*"would" + 0.007*"said" + 0.006*"new" + 0.006*"presid" + 0.006*"one" + 0.005*"peopl" + 0.005*"govern" + 0.005*"time" + 0.004*"work" + 0.004*"polit"'),
 (1,
  '0.047*"secretari" + 0.044*"depart" + 0.044*"state" + 0.040*"offic" + 0.024*"meet" + 0.019*"room" + 0.015*"hous" + 0.014*"arriv" + 0.013*"rout" + 0.012*"privat"'),
 (2,
  '0.021*"call" + 0.017*"h" + 0.013*"see" + 0.013*"get" + 0.011*"work" + 0.010*"w" + 0.010*"know" + 0.009*"tomorrow" + 0.009*"go" + 0.009*"today"')]

In [215]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
for i, words in topics_matrix:
    print([w for w, p in words], '\n')

['would', 'said', 'new', 'presid', 'one', 'peopl', 'govern', 'time', 'work', 'polit', 'us', 'like', 'also', 'support', 'american', 'clinton', 'secur', 'unit', 'nation', 'first'] 

['secretari', 'depart', 'state', 'offic', 'meet', 'room', 'hous', 'arriv', 'rout', 'privat', 'confer', 'time', 'resid', 'inform', 'subject', 'date', 'staff', 'foreign', 'sensit', 'case'] 

['call', 'h', 'see', 'get', 'work', 'w', 'know', 'tomorrow', 'go', 'today', 'also', 'talk', 'good', 'thank', 'would', 'let', 'want', 'back', 'like', 'time'] 

